<a href="https://colab.research.google.com/github/armandossrecife/piloto/blob/main/analise_designite_jackson_databind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Checa ambiente

In [1]:
!java --version

openjdk 11.0.20.1 2023-08-24
OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)


In [2]:
!python3 --version

Python 3.10.12


In [3]:
!sudo apt install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (365 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 120893 files and directories currently install

# Funcões

In [11]:
import pandas as pd
import os
import requests

def checa_indice_language(relacao_linguagens):
  indice=-1
  for i, each in enumerate(relacao_linguagens):
    if "Language" in each:
      indice = i
      break
  return indice

def instala_cloc():
  try:
    comando = !sudo apt-get install cloc
    print(comando)
  except Exception as e:
    print(f'Erro: {str(e)}')

def clona_repositorio(url):
  try:
    comando = f'git clone {url}'
    print(f'Clonando o repositório {url}')
    print(comando)
    os.system(comando)
    print('concluido com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def get_primeiro_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | tail -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_commit_tag(tag):
  try:
    os.environ['TAG'] = tag
    comando = !cd jackson-databind && git rev-list -n 1 $TAG
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_ultimo_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | head -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def cria_pasta_n_commit(repositorio, pasta_n_commit):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Cria uma nova pasta para referenciar apenas os dados do 1o commit
    comando = !cp -R $REPOSITORIO_ORIGINAL $REPOSITORIO_INICIAL
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def checkout_n_commit(pasta_n_commit, n_commit):
  try:
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    os.environ['PRIMEIRO_COMMIT'] = n_commit
    # git checkout $( git rev-list --all | tail -n 1 )
    comando1 = "cd $REPOSITORIO_INICIAL && git checkout $PRIMEIRO_COMMIT"
    os.system(comando1)
    print(f'Checkout do repositorio no commit {n_commit} realizado com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def gera_df_info_repositorio(pasta_n_commit):
  try:
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Quantidade de arquivos na pasta nome_repositorio
    qtd_arquivos = !find $REPOSITORIO_INICIAL -type f | wc -l
    # Quantidade de commits do repositorio nome_repositorio
    qtd_commits = !cd $REPOSITORIO_INICIAL && git log --oneline | grep -c .
    # Tamanho (MB) da pasta nome_repositorio
    tamanho_pasta_mb = !du -hs --block-size=1M $REPOSITORIO_INICIAL | cut -f1
    # Quantidade de colaboradores (devs que commitaram pelo menos uma vez no projeto nome_repositorio)
    qtd_colaboradores_commits = !cd $REPOSITORIO_INICIAL && git log --pretty=format:"%ae" | sort -u | grep -v "noreply.github.com" | wc -l
    # Data do primeiro commit do projeto nome_repositorio
    data_primeiro_commit =!cd $REPOSITORIO_INICIAL && git log --reverse --pretty=format:%ad --date=short | head -1
    # Data do ultimo commit do projeto nome_repositorio
    data_ultimo_commit = !cd $REPOSITORIO_INICIAL && git log -1 --pretty=format:%ad --date=short

    dict_info_repo = {
      'nome':pasta_n_commit,
      'qtd_arquivos':qtd_arquivos,
      'qtd_commits':qtd_commits,
      'tamanho_pasta_mb':tamanho_pasta_mb,
      'qtd_colaboradores_commits':qtd_colaboradores_commits,
      'data_primeiro_commit':data_primeiro_commit,
      'data_ultimo_commit':data_ultimo_commit,
    }

    df_info_repo = pd.DataFrame(dict_info_repo)
  except Exception as e:
    print(f'Erro: {str(e)}')
  return df_info_repo

def gera_df_dados(pasta_n_commit):
  lista_linguagens, lista_files, lista_blank, lista_comment, lista_code = list(), list(), list(), list(), list()
  l_temp = []
  vl_temp = []
  os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
  try:
    # Detalha os tipos de arquivos que aparecem no repositório do nome_repositorio
    relacao_linguagens = !cd $REPOSITORIO_INICIAL && cloc .
    #relacao_linguagens[checa_indice_language(relacao_linguagens):]
    linguagens_encontradas = relacao_linguagens[(checa_indice_language(relacao_linguagens)+2):]
    for linha in linguagens_encontradas:
      elementos = linha.split(' ')
      novos_elementos = []
      linguagem_encontrada = []
      valores_da_linguagem = []
      for item in elementos:
        if item != '':
          novos_elementos.append(item)
      linguagem_encontrada.append(novos_elementos[:-4])
      valores_da_linguagem.append(novos_elementos[-4:])
      l_temp = l_temp + linguagem_encontrada
      vl_temp = vl_temp + valores_da_linguagem

    l_temp = l_temp[:-3]
    vl_temp = vl_temp[:-3]
    for i,each in enumerate(vl_temp):
      lista_files.append(each[0])
      lista_blank.append(each[1])
      lista_comment.append(each[2])
      lista_code.append(each[3])
  except Exception as e:
    print(f'Erro: {str(e)}')
  try:
    dict_dados_linguagens = {
    'linguagens':l_temp,
    'files':lista_files,
    'blank':lista_blank,
    'comment':lista_comment,
    'code':lista_code
    }
    df_dados_linguagens = pd.DataFrame(dict_dados_linguagens)
    df_dados_linguagens['files'] = df_dados_linguagens['files'].apply(lambda x: int(x))
    df_dados_linguagens['blank'] = df_dados_linguagens['blank'].apply(lambda x: int(x))
    df_dados_linguagens['comment'] = df_dados_linguagens['comment'].apply(lambda x: int(x))
    df_dados_linguagens['code'] = df_dados_linguagens['code'].apply(lambda x: int(x))
  except Exception as e2:
    print(f'Erro: {str(e2)}')

  return df_dados_linguagens

def gera_df_dados_repositorios(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'descricao':l_cf,
        'dados_c1':c_c1,
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

def gera_df_dados_repositorios2(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

In [12]:
def download_designite():
  try:
    url = 'https://www.designite-tools.com/static/download/DJC/DesigniteJava.jar'  # replace with the URL of the file you want to download
    filename = 'DesigniteJava.jar'  # replace with the desired filename to save the downloaded file
    response = requests.get(url)

    with open(filename, 'wb') as f:
        f.write(response.content)

    print(f'{filename} downloaded successfully!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def run_designite(tag, pasta_testes):
  try:
    os.environ['TESTES'] = pasta_testes
    os.environ['JACKSON_DATABIND'] = tag + '/src/main/java'
    cria_pasta_testes = !mkdir $TESTES
    print(cria_pasta_testes)
    executa_designite = !java -jar DesigniteJava.jar -i $JACKSON_DATABIND -o $TESTES
    print(executa_designite)
  except Exception as e:
    print(f'Erro: {str(e)}')

def gera_df_design_smells(pasta_testes, lista):
  path_file = pasta_testes + '/' + 'designCodeSmells.csv'
  df_ds = pd.read_csv(path_file)
  elemento = pasta_testes,df_ds
  lista.append(elemento)
  return df_ds

In [13]:
def is_valid_url(url):
    try:
        response = requests.head(url)
        return response.status_code in range(200, 300)  # Check if status code is in the 2xx range
    except requests.exceptions.RequestException:
        return False  # Any error or exception indicates an invalid or non-existent URL

In [14]:
import re

def get_elements_with_pattern(strings_list, pattern):
    matching_elements = [s for s in strings_list if re.search(pattern, s)]
    return matching_elements

# Example list of strings
strings_list = ['gson-parent-1.0', 'gson-parent-2.0', 'gson-child-1.0', 'gson-parent-3.0']
pattern_to_match = r'gson-parent'

result = get_elements_with_pattern(strings_list, pattern_to_match)
print(result)

['gson-parent-1.0', 'gson-parent-2.0', 'gson-parent-3.0']


# Análise Designite

In [8]:
import requests

url = 'https://www.designite-tools.com/static/download/DJC/DesigniteJava.jar'  # replace with the URL of the file you want to download
filename = 'DesigniteJava.jar'  # replace with the desired filename to save the downloaded file
response = requests.get(url)

with open(filename, 'wb') as f:
    f.write(response.content)

print(f'{filename} downloaded successfully!')

DesigniteJava.jar downloaded successfully!


In [9]:
!git clone https://github.com/FasterXML/jackson-databind.git

Cloning into 'jackson-databind'...
remote: Enumerating objects: 162247, done.
remote: Counting objects: 100% (30817/30817), done.
remote: Compressing objects: 100% (1569/1569), done.
remote: Total 162247 (delta 29100), reused 30015 (delta 28564), pack-reused 131430
Receiving objects: 100% (162247/162247), 72.52 MiB | 21.25 MiB/s, done.
Resolving deltas: 100% (88920/88920), done.


In [10]:
!tree jackson-databind -L 2

jackson-databind
├── attic
│   ├── ContainerBuilder.java
│   └── README.md
├── create-test-report.sh
├── docs
│   ├── javadoc
│   └── README.md
├── LICENSE
├── mvnw
├── mvnw.cmd
├── pom.xml
├── profile.sh
├── README.md
├── release-notes
│   ├── CREDITS-2.x
│   └── VERSION-2.x
├── run.sh
├── SECURITY.md
└── src
    ├── main
    ├── moditect
    ├── test
    ├── test-jdk14
    ├── test-jdk17
    └── test-jdk21

11 directories, 14 files


### Pega a lista de tags

In [16]:
lista_tags_jackson = !cd jackson-databind && git tag
lista_tags_jackson

['2.2.0c',
 '2.6.0-rc3b',
 'jackson-databind-2.0.0',
 'jackson-databind-2.0.0-RC1',
 'jackson-databind-2.0.0-RC2',
 'jackson-databind-2.0.0-RC3',
 'jackson-databind-2.0.1',
 'jackson-databind-2.0.2',
 'jackson-databind-2.0.4',
 'jackson-databind-2.0.5',
 'jackson-databind-2.0.6',
 'jackson-databind-2.1.0',
 'jackson-databind-2.1.1',
 'jackson-databind-2.1.2',
 'jackson-databind-2.1.3',
 'jackson-databind-2.1.4',
 'jackson-databind-2.1.5',
 'jackson-databind-2.10.0',
 'jackson-databind-2.10.0.pr1',
 'jackson-databind-2.10.0.pr2',
 'jackson-databind-2.10.0.pr3',
 'jackson-databind-2.10.1',
 'jackson-databind-2.10.2',
 'jackson-databind-2.10.3',
 'jackson-databind-2.10.4',
 'jackson-databind-2.10.5',
 'jackson-databind-2.10.5.1',
 'jackson-databind-2.11.0',
 'jackson-databind-2.11.0.rc1',
 'jackson-databind-2.11.1',
 'jackson-databind-2.11.2',
 'jackson-databind-2.11.3',
 'jackson-databind-2.11.4',
 'jackson-databind-2.12.0',
 'jackson-databind-2.12.0-rc1',
 'jackson-databind-2.12.0-rc2',

In [17]:
pattern_to_extract = r'-2.11'
get_elements_with_pattern(lista_tags_jackson, pattern_to_extract)

['jackson-databind-2.11.0',
 'jackson-databind-2.11.0.rc1',
 'jackson-databind-2.11.1',
 'jackson-databind-2.11.2',
 'jackson-databind-2.11.3',
 'jackson-databind-2.11.4']

In [18]:
pattern_to_extract = r'-2.15'
get_elements_with_pattern(lista_tags_jackson, pattern_to_extract)

['jackson-databind-2.15.0',
 'jackson-databind-2.15.0-rc1',
 'jackson-databind-2.15.0-rc2',
 'jackson-databind-2.15.0-rc3',
 'jackson-databind-2.15.1',
 'jackson-databind-2.15.2']

# Análise da tag jackson-databind-2.11.0

In [19]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
commit_tag_jd_2_11_0 = get_commit_tag(tag='jackson-databind-2.11.0')

In [20]:
cria_pasta_n_commit(repositorio='jackson-databind', pasta_n_commit='commit_tag_jd_2_11_0')

[]

In [22]:
!ls commit_tag_jd_2_11_0/src

main  moditect	test  test-jdk14  test-jdk17  test-jdk21


In [23]:
checkout_n_commit('commit_tag_jd_2_11_0', commit_tag_jd_2_11_0[0])

Checkout do repositorio no commit 276687428a5fd9c1f00f0f3231daff2f8c2bcfa3 realizado com sucesso!


In [26]:
ls -liath commit_tag_jd_2_11_0/src/main/java

total 12K
4064931 drwxr-xr-x 4 root root 4.0K Sep  1 19:18 ../
4064932 drwxr-xr-x 3 root root 4.0K Sep  1 19:18 ./
4064933 drwxr-xr-x 3 root root 4.0K Sep  1 19:18 com/


In [27]:
!java -jar DesigniteJava.jar -i commit_tag_jd_2_11_0/src/main/java -o commit_tag_jd_2_11_0

Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.


In [28]:
!ls -liat commit_tag_jd_2_11_0/designCodeSmells.csv

4064847 -rw-r--r-- 1 root root 63700 Sep  1 19:21 commit_tag_jd_2_11_0/designCodeSmells.csv


In [29]:
df_design_smells = pd.read_csv('commit_tag_jd_2_11_0/designCodeSmells.csv')

In [30]:
df_design_smells

,Project Name,Package Name,Type Name,Code Smell
0,java,com.fasterxml.jackson.databind,DatabindContext,Cyclic-Dependent Modularization
1,java,com.fasterxml.jackson.databind,DatabindContext,Insufficient Modularization
2,java,com.fasterxml.jackson.databind,BeanProperty,Deficient Encapsulation
3,java,com.fasterxml.jackson.databind,BeanProperty,Rebellious Hierarchy
4,java,com.fasterxml.jackson.databind,BeanProperty,Cyclic-Dependent Modularization
...,...,...,...,...
732,java,com.fasterxml.jackson.databind.type,TypeParser,Cyclic-Dependent Modularization
733,java,com.fasterxml.jackson.databind.type,TypeBindings,Cyclic-Dependent Modularization
734,java,com.fasterxml.jackson.databind.type,TypeBindings,Insufficient Modularization
735,java,com.fasterxml.jackson.databind.jsonschema,SchemaAware,Unutilized Abstraction


In [31]:
df_design_smells['Type Name'].unique()

array(['DatabindContext', 'BeanProperty', 'Std', 'Bogus', 'ObjectWriter',
       'GeneratorSettings', 'Prefetch', 'RuntimeJsonMappingException',
       'JsonNode', 'InjectableValues', 'DeserializationConfig',
       'JsonMappingException', 'SerializationConfig', 'JsonSerializable',
       'Base', 'AnnotationIntrospector', 'KeyDeserializer', 'None',
       'ObjectReader', 'PropertyMetadata', 'MergeInfo', 'BeanDescription',
       'JsonDeserializer', 'JavaType', 'AbstractTypeResolver', 'Module',
       'SetupContext', 'PropertyNamingStrategy',
       'PropertyNamingStrategyBase', 'SnakeCaseStrategy',
       'UpperCamelCaseStrategy', 'LowerCaseStrategy', 'KebabCaseStrategy',
       'LowerDotCaseStrategy', 'LowerCaseWithUnderscoresStrategy',
       'PascalCaseStrategy', 'DeserializationContext', 'SequenceWriter',
       'SerializerProvider', 'ObjectMapper', 'DefaultTypeResolverBuilder',
       'JsonSerializer', 'PropertyName', 'TreeTraversingParser',
       'ContainerNode', 'DoubleNode', '

In [33]:
# Esta lista eh oriunda da analise AMLOC x FOC e das analises dos 3o quartis AMLOC e FOC
df_jackson_databind_amloc_foc = pd.read_csv('https://raw.githubusercontent.com/armandossrecife/teste/main/jackson-databind_arquivos_criticos.csv')
df_jackson_databind_amloc_foc

lista_classes_criticas = df_jackson_databind_amloc_foc.nome.unique().tolist()
lista_classes_criticas

['AnnotationIntrospector.java',
 'AnnotationIntrospectorPair.java',
 'BasicDeserializerFactory.java',
 'BeanDeserializer.java',
 'BeanDeserializerBase.java',
 'ClassUtil.java',
 'DeserializationFeature.java',
 'EnumDeserializer.java',
 'EnumResolver.java',
 'EnumValues.java',
 'FactoryBasedEnumDeserializer.java',
 'JacksonAnnotationIntrospector.java',
 'JsonNode.java',
 'MapperFeature.java',
 'NumberDeserializers.java',
 'ObjectMapper.java',
 'ObjectNode.java',
 'POJOPropertiesCollector.java',
 'StdDeserializer.java',
 'StdTypeResolverBuilder.java',
 'TokenBuffer.java']

In [36]:
lista_classes_criticas_only_name = []
for each in lista_classes_criticas:
  each = each.replace('.java', '')
  lista_classes_criticas_only_name.append(each)
lista_classes_criticas_only_name

['AnnotationIntrospector',
 'AnnotationIntrospectorPair',
 'BasicDeserializerFactory',
 'BeanDeserializer',
 'BeanDeserializerBase',
 'ClassUtil',
 'DeserializationFeature',
 'EnumDeserializer',
 'EnumResolver',
 'EnumValues',
 'FactoryBasedEnumDeserializer',
 'JacksonAnnotationIntrospector',
 'JsonNode',
 'MapperFeature',
 'NumberDeserializers',
 'ObjectMapper',
 'ObjectNode',
 'POJOPropertiesCollector',
 'StdDeserializer',
 'StdTypeResolverBuilder',
 'TokenBuffer']

In [39]:
pesquisa = df_design_smells['Type Name'].str.contains('|'.join(lista_classes_criticas_only_name))

In [40]:
df_critical_classes = df_design_smells[pesquisa]
df_critical_classes

,Project Name,Package Name,Type Name,Code Smell
15,java,com.fasterxml.jackson.databind,JsonNode,Unutilized Abstraction
16,java,com.fasterxml.jackson.databind,JsonNode,Cyclic-Dependent Modularization
17,java,com.fasterxml.jackson.databind,JsonNode,Insufficient Modularization
28,java,com.fasterxml.jackson.databind,AnnotationIntrospector,Cyclic-Dependent Modularization
29,java,com.fasterxml.jackson.databind,AnnotationIntrospector,Insufficient Modularization
72,java,com.fasterxml.jackson.databind,ObjectMapper,Cyclic-Dependent Modularization
73,java,com.fasterxml.jackson.databind,ObjectMapper,Insufficient Modularization
94,java,com.fasterxml.jackson.databind.node,JsonNodeFactory,Unutilized Abstraction
95,java,com.fasterxml.jackson.databind.node,JsonNodeFactory,Deficient Encapsulation
96,java,com.fasterxml.jackson.databind.node,JsonNodeFactory,Cyclic-Dependent Modularization


In [41]:
df_critical_classes.columns = ['ProjectName', 'PackageName', 'TypeName', 'CodeSmell']
df_critical_classes = df_critical_classes.query("CodeSmell=='Cyclic-Dependent Modularization' or CodeSmell=='Hub-like Modularization'")
df_critical_classes

,ProjectName,PackageName,TypeName,CodeSmell
16,java,com.fasterxml.jackson.databind,JsonNode,Cyclic-Dependent Modularization
28,java,com.fasterxml.jackson.databind,AnnotationIntrospector,Cyclic-Dependent Modularization
72,java,com.fasterxml.jackson.databind,ObjectMapper,Cyclic-Dependent Modularization
96,java,com.fasterxml.jackson.databind.node,JsonNodeFactory,Cyclic-Dependent Modularization
99,java,com.fasterxml.jackson.databind.node,BaseJsonNode,Cyclic-Dependent Modularization
134,java,com.fasterxml.jackson.databind.node,ObjectNode,Cyclic-Dependent Modularization
185,java,com.fasterxml.jackson.databind.introspect,AnnotationIntrospectorPair,Cyclic-Dependent Modularization
230,java,com.fasterxml.jackson.databind.deser,BeanDeserializerModifier,Cyclic-Dependent Modularization
231,java,com.fasterxml.jackson.databind.deser,BeanDeserializerBuilder,Cyclic-Dependent Modularization
251,java,com.fasterxml.jackson.databind.deser,BeanDeserializerFactory,Cyclic-Dependent Modularization


In [42]:
df_analise_designite_critical_classes = df_critical_classes.sort_values(by='TypeName')
df_analise_designite_critical_classes.to_csv('commit_tag_jd_2_11_0/analise_designite_classes_criticas.csv', index=False)
df_design_smells.to_csv('commit_tag_jd_2_11_0/analise_designite.csv', index=False)

In [43]:
!cd commit_tag_jd_2_11_0 && zip ds_commit_tag_jd_2_11_0.zip *.csv

  adding: analise_designite_classes_criticas.csv (deflated 80%)
  adding: analise_designite.csv (deflated 93%)
  adding: designCodeSmells.csv (deflated 93%)
  adding: implementationCodeSmells.csv (deflated 94%)
  adding: methodMetrics.csv (deflated 92%)
  adding: typeMetrics.csv (deflated 80%)


# Análise da tag jackson-databind-2.15.2


In [44]:
commit_tag_jackson_databind_2_15_2 = get_commit_tag(tag='jackson-databind-2.15.2')

In [46]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
cria_pasta_n_commit(repositorio='jackson-databind', pasta_n_commit='commit_tag_jackson_databind_2_15_2')

[]

In [47]:
checkout_n_commit('commit_tag_jackson_databind_2_15_2', commit_tag_jackson_databind_2_15_2[0])

Checkout do repositorio no commit 74a57b2f6cd3b6e0fe2a61ed16734f15ba2abc95 realizado com sucesso!


In [48]:
!java -jar DesigniteJava.jar -i commit_tag_jackson_databind_2_15_2/src/main/java -o commit_tag_jackson_databind_2_15_2

Parsing the source code ...
Resolving symbols...
Extracting metrics...
Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.
Extracting code smells...
Ancestors analysis skipped due to memory overflow.Ancestors analysis skipped due to memory overflow.Ancestors analysis skipped due to memory overflow.
Ancestors analysis skipped due to memory overflow.Ancestors analysis skipped due to memory overflow.Ancestors analysis skipped due to memory overflow.
Ancestors analysis skipped due to memory overflow.Ancestors analysis skipped due to memory overflow.Ancestors analysis skipped due to memory overflow.Ancestors analysis skipped due to memory overflow.Ancestors analysis skipped due to memory overflow.
Ancestors analysis skipped due to memory overflow.Ancestors analysis skipped due to memory overflow.Ancestors analysis skipped due to memory overflow.Ancestors 

In [49]:
df_design_smells_final = pd.read_csv('commit_tag_jackson_databind_2_15_2/designCodeSmells.csv')

In [50]:
df_design_smells_final

,Project Name,Package Name,Type Name,Code Smell
0,java,com.fasterxml.jackson.databind,DatabindContext,Cyclic-Dependent Modularization
1,java,com.fasterxml.jackson.databind,DatabindContext,Insufficient Modularization
2,java,com.fasterxml.jackson.databind,BeanProperty,Deficient Encapsulation
3,java,com.fasterxml.jackson.databind,BeanProperty,Rebellious Hierarchy
4,java,com.fasterxml.jackson.databind,BeanProperty,Cyclic-Dependent Modularization
...,...,...,...,...
849,java,com.fasterxml.jackson.databind.type,IdentityEqualityType,Broken Hierarchy
850,java,com.fasterxml.jackson.databind.type,TypeBindings,Cyclic-Dependent Modularization
851,java,com.fasterxml.jackson.databind.type,TypeBindings,Insufficient Modularization
852,java,com.fasterxml.jackson.databind.jsonschema,SchemaAware,Unutilized Abstraction


In [51]:
pesquisa_final = df_design_smells_final['Type Name'].str.contains('|'.join(lista_classes_criticas_only_name))

In [52]:
df_critical_classes_final = df_design_smells_final[pesquisa_final]
df_critical_classes_final.columns = ['ProjectName', 'PackageName', 'TypeName', 'CodeSmell']
df_critical_classes_final = df_critical_classes_final.query("CodeSmell=='Cyclic-Dependent Modularization' or CodeSmell=='Hub-like Modularization'")

In [53]:
df_critical_classes_final

,ProjectName,PackageName,TypeName,CodeSmell
41,java,com.fasterxml.jackson.databind,JsonNode,Cyclic-Dependent Modularization
53,java,com.fasterxml.jackson.databind,AnnotationIntrospector,Cyclic-Dependent Modularization
99,java,com.fasterxml.jackson.databind,ObjectMapper,Cyclic-Dependent Modularization
125,java,com.fasterxml.jackson.databind.node,JsonNodeFactory,Cyclic-Dependent Modularization
128,java,com.fasterxml.jackson.databind.node,BaseJsonNode,Cyclic-Dependent Modularization
168,java,com.fasterxml.jackson.databind.node,ObjectNode,Cyclic-Dependent Modularization
219,java,com.fasterxml.jackson.databind.introspect,POJOPropertiesCollector,Cyclic-Dependent Modularization
233,java,com.fasterxml.jackson.databind.introspect,AnnotationIntrospectorPair,Cyclic-Dependent Modularization
280,java,com.fasterxml.jackson.databind.deser,BeanDeserializerModifier,Cyclic-Dependent Modularization
281,java,com.fasterxml.jackson.databind.deser,BeanDeserializerBuilder,Cyclic-Dependent Modularization


In [54]:
df_analise_designite_critical_classes_final = df_critical_classes_final.sort_values(by='TypeName')
df_analise_designite_critical_classes_final.to_csv('commit_tag_jackson_databind_2_15_2/analise_designite_classes_criticas.csv', index=False)
df_design_smells_final.to_csv('commit_tag_jackson_databind_2_15_2/analise_designite.csv', index=False)

In [55]:
!cd commit_tag_jackson_databind_2_15_2 && zip ds_commit_tag_jackson_databind_2_15_2.zip *.csv

  adding: analise_designite_classes_criticas.csv (deflated 83%)
  adding: analise_designite.csv (deflated 93%)
  adding: designCodeSmells.csv (deflated 93%)
  adding: implementationCodeSmells.csv (deflated 93%)
  adding: methodMetrics.csv (deflated 92%)
  adding: typeMetrics.csv (deflated 80%)
